In [1]:
# ==========================================================
# Eq. (9) — BAU-normalized transition cost (EXPORTS REFACTORED ONLY)
# - Calculations and plotting identical to your current workflow.
# - Adds standardized exports for Streamlit (PNG + CSV wide + CSV long).
# ==========================================================
import os, re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from pathlib import Path

# --------------------------- CONFIG ---------------------------
ROOT = os.path.expanduser("~/Documents/Summer_intern")
EMISSIONS_XLSX = os.path.join(ROOT, "NGFS_GCAM_Carbon_Emissions_Sectors.xlsx")
PRICES_XLSX = os.path.join(ROOT, "NGFS_GCAM_Price_Carbons.xlsx")

OUTDIR = os.path.join(ROOT, "outputs"); os.makedirs(OUTDIR, exist_ok=True)

# Legacy outputs (kept for backward compatibility)
PDF_OUT_MAIN = os.path.join(OUTDIR, "NGFS_Heatmaps_Transition_Cost.pdf")
PNG_DIR_MAIN = os.path.join(OUTDIR, "NGFS_Eq9_png"); os.makedirs(PNG_DIR_MAIN, exist_ok=True)

PDF_OUT_20 = os.path.join(OUTDIR, "NGFS_Heatmaps_Transition_Cost_BAU20_r2pct.pdf")
PNG_DIR_20 = os.path.join(OUTDIR, "NGFS_Eq9_BAU20_r2pct_png"); os.makedirs(PNG_DIR_20, exist_ok=True)

# New standardized tree for Streamlit (added; calculations unchanged)
STD_ROOT = Path(ROOT) / "outputs" / "transition"  # base for standardized exports

BAU_SCEN = "Current Policies"  # b
NZ_SCEN  = "Net Zero 2050"     # z

REQUESTED_REGIONS = [
    "USA","EU-15","Europe non EU","Japan","China","India","South Korea",
    "Canada","Australia-NZ","Taiwan","Mexico","Russia","Brazil","South Africa","Argentina"
]
REGION_MAP = {"Europe non EU": "Europe_Non_EU", "Australia-NZ": "Australia_NZ"}

# Fixed BAU carbon prices ($/tCO2) — strict coverage for MAIN run
REGIONAL_BAU_FIXED = {
    "USA": 23.0, "EU-15": 70.0, "Europe_Non_EU": 60.0, "Japan": 4.0,
    "China": 11.8, "India": 2.0, "South Korea": 6.0, "Canada": 66.0,
    "Australia_NZ": 26.0, "Taiwan": 9.0, "Mexico": 3.9, "Russia": 2.0,
    "Brazil": 2.0, "South Africa": 12.8, "Argentina": 5.3,
}

HORIZONS = [2030, 2050]
T0_ANCHOR = 2020
DISCOUNT_RATES_MAIN = [0.02, 0.00]  # main: r=2% and r=0%
DISCOUNT_RATES_20   = [0.02]        # BAU=$20 run: r=2% only

# ---- NGFS sector names & order (10 columns) ----
SECTOR_ORDER_FIXED = [
    "Supply","Other Energy Supply","Electricity","Industry","Other Industry",
    "Chemicals","Cement","Transportation","AFOLU","Other",
]
SUPPLY_COMPONENTS = ["Electricity", "Other Energy Supply"]

# --------------------------- HELPERS ---------------------------
def read_sheet(path, sheetname="data"):
    if not os.path.exists(path): raise FileNotFoundError(path)
    xls = pd.ExcelFile(path)
    if sheetname not in xls.sheet_names: raise ValueError(f"'{sheetname}' not in {xls.sheet_names}")
    return pd.read_excel(xls, sheet_name=sheetname)

def longify_years(df, id_vars):
    year_cols = [c for c in df.columns if str(c).isdigit()]
    out = df.melt(id_vars=id_vars, value_vars=year_cols, var_name="Year", value_name="Value")
    out["Year"] = out["Year"].astype(int)
    return out.dropna(subset=["Value"])

def filter_kyoto(df_long):
    return df_long[df_long["Variable"].astype(str).str.startswith("Emissions|Kyoto Gases|")].copy()

_CANON = {
    "supply":"Supply","other energy supply":"Other Energy Supply",
    "electricity":"Electricity","power":"Electricity",
    "industry":"Industry","other industry":"Other Industry",
    "chemicals":"Chemicals","cement":"Cement",
    "transportation":"Transportation","transport":"Transportation",
    "afolu":"AFOLU","other":"Other",
}
def ngfs_sector(variable: str) -> str:
    last = str(variable).split("|")[-1].strip().lower()
    return _CANON.get(last, "Other")

def resolve_regions(req, rmap, available):
    mapped  = [rmap.get(r, r) for r in req]
    present = [r for r in mapped if r in available]
    reverse = {rmap.get(k, k): k for k in req}
    return present, reverse

def discount_factors(years, r, t0=T0_ANCHOR):
    years = np.asarray(years, dtype=int)
    return pd.Series((1.0 + r)**(-(years - t0)), index=years)

def per_heatmap_vmax(matrix, p=97):
    vals = matrix.values.ravel().astype(float)
    vals = vals[np.isfinite(vals)]
    return max(np.nanpercentile(vals, p), 1e-12) if vals.size else 1.0

def slugify(s: str) -> str:
    return re.sub(r"[^A-Za-z0-9._-]+", "_", s).strip("_")

# ---------- NEW: standardized export path helpers (exports only) ----------
def std_paths(horizon_end: int, r_disc: float, mode: str):
    """Return standardized data/png paths for Streamlit.
       mode ∈ {"eq9_regional", "eq9_bau20"}.
    """
    horizon_str = f"2020-{horizon_end}"
    rtag = f"r{int(round(r_disc*100))}"  # r2 or r0
    base = STD_ROOT / horizon_str / mode / rtag
    data_dir = base / "data"
    png_dir  = base / "png"
    data_dir.mkdir(parents=True, exist_ok=True)
    png_dir.mkdir(parents=True, exist_ok=True)
    return {
        "data_wide": data_dir / "heatmap_data.csv",
        "data_long": data_dir / "heatmap_data_long.csv",
        "png":       png_dir  / "heatmap.png",
    }

# --------------------------- LOAD DATA ---------------------------
em_raw = read_sheet(EMISSIONS_XLSX, "data")
em_raw["Region"] = em_raw["Region"].astype(str).str.split("|").str[-1]
em_long = longify_years(em_raw, ["Model","Scenario","Region","Variable","Unit"])
em_long = filter_kyoto(em_long)

pr_raw = read_sheet(PRICES_XLSX, "data")
pr_raw["Region"] = pr_raw["Region"].astype(str).str.split("|").str[-1]
pr_long = longify_years(pr_raw, ["Model","Scenario","Region","Variable","Unit"])

# Regions intersection & order
avail_regions = set(em_long["Region"].unique()) & set(pr_long["Region"].unique())
regions_order, reverse = resolve_regions(REQUESTED_REGIONS, REGION_MAP, avail_regions)
em_long = em_long[em_long["Region"].isin(regions_order)].copy()
pr_long = pr_long[pr_long["Region"].isin(regions_order)].copy()

# Strict BAU fixed-price coverage for MAIN run
missing_bau = [r for r in regions_order if r not in REGIONAL_BAU_FIXED]
if missing_bau:
    raise ValueError(f"Missing BAU fixed price for regions: {missing_bau}")
nonpos_bau = [r for r,v in REGIONAL_BAU_FIXED.items() if r in regions_order and (v is None or v <= 0)]
if nonpos_bau:
    raise ValueError(f"Non-positive BAU fixed price for regions: {nonpos_bau}")

# --------------------------- PREP SERIES ---------------------------
def pivot_emissions(df, scenario, year_max):
    d = df[(df["Scenario"]==scenario) & (df["Year"]<=year_max)].copy()
    d["Sector"] = d["Variable"].apply(ngfs_sector)
    return d.rename(columns={"Value":"E"})

def nz_prices_region(df, year_max):
    d = df[(df["Scenario"]==NZ_SCEN) & (df["Year"]<=year_max) &
           (df["Variable"].astype(str).str.fullmatch(r"Price\|Carbon"))][["Region","Year","Value"]]
    if d.empty:
        raise ValueError("No region-level NZ 'Price|Carbon' found in PRICES_XLSX.")
    return d.rename(columns={"Value":"Pz_reg"})

# --------------------------- METRIC (Eq. 9) ---------------------------
def compute_eq9_matrix(H, r_disc, bau_price_spec):
    """
    bau_price_spec: dict {Region -> price} or a float (constant for all regions).
    Returns matrix (Region x Sector) of Eq9_Ctilde.
    """
    Eb = pivot_emissions(em_long, BAU_SCEN, H).rename(columns={"E":"E_b"})
    Ez = pivot_emissions(em_long, NZ_SCEN, H).rename(columns={"E":"E_z"})
    key = ["Region","Variable","Unit","Year","Sector"]
    m = Eb[key+["E_b"]].merge(Ez[key+["E_z"]], on=key, how="inner")

    # NZ region-level prices
    Pz = nz_prices_region(pr_long, H)
    m = m.merge(Pz, on=["Region","Year"], how="left")

    # Fixed BAU price per region
    if isinstance(bau_price_spec, dict):
        m["Pb_den"] = m["Region"].map(bau_price_spec).astype(float)
        if m["Pb_den"].isna().any():
            bad = m.loc[m["Pb_den"].isna(),"Region"].unique().tolist()
            raise ValueError(f"BAU fixed price unexpectedly missing after mapping for: {bad}")
    else:
        m["Pb_den"] = float(bau_price_spec)

    # (E_b - E_z)_+
    m["Excess_pos"] = np.clip(m["E_b"] - m["E_z"], 0.0, None)

    # Discount
    years = sorted(m["Year"].unique())
    D = discount_factors(years, r=r_disc, t0=T0_ANCHOR)
    m = m.merge(D.rename("D").rename_axis("Year").reset_index(), on="Year", how="left")

    # Per-row contributions
    m["Num_t"] = m["D"] * m["Pz_reg"] * m["Excess_pos"]
    m["Den_t"] = m["D"] * m["Pb_den"] * m["E_b"]

    # Group by (Region, Sector)
    grp = (m.groupby(["Region","Sector"], as_index=False)[["Num_t","Den_t"]].sum())

    # Synthesize 'Supply' if components exist
    present = set(grp["Sector"].unique())
    if ("Supply" not in present) and all(s in present for s in SUPPLY_COMPONENTS):
        supply = (grp[grp["Sector"].isin(SUPPLY_COMPONENTS)]
                  .groupby("Region", as_index=False)[["Num_t","Den_t"]].sum())
        supply["Sector"] = "Supply"
        grp = pd.concat([grp, supply], ignore_index=True)

    # Final ratio
    grp["Eq9_Ctilde"] = np.where(grp["Den_t"] > 0, grp["Num_t"]/grp["Den_t"], np.nan)

    # Pivot to matrix with fixed NGFS order
    mat = grp.pivot(index="Region", columns="Sector", values="Eq9_Ctilde")
    mat = mat.reindex(index=regions_order)
    for col in SECTOR_ORDER_FIXED:
        if col not in mat.columns:
            mat[col] = np.nan
    mat = mat[SECTOR_ORDER_FIXED]
    return mat

# --------------------------- PLOTTING ---------------------------
def plot_heatmap(mat, title, rotate_xticks=True):
    import seaborn as sns
    fig, ax = plt.subplots(figsize=(0.5*max(8, mat.shape[1]) + 6,
                                    0.45*max(6, mat.shape[0]) + 3))
    cm = plt.get_cmap("RdYlGn_r").copy()
    try: cm.set_bad('#e6e6e6')
    except: pass
    vmax = per_heatmap_vmax(mat, p=97)
    sns.heatmap(mat.clip(lower=0, upper=vmax), ax=ax, cmap=cm,
                vmin=0, vmax=vmax, mask=np.isnan(mat.values),
                linewidths=0.2, linecolor='white', cbar=False)
    if rotate_xticks:
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right",
                           rotation_mode="anchor", fontsize=9)
    ax.set_title(title, fontsize=11); ax.set_xlabel("Sector"); ax.set_ylabel("Region")
    im = ax.collections[0]
    cbar = fig.colorbar(im, ax=ax, fraction=0.046, pad=0.02)
    cbar.set_label(r"$\tilde{C}^{z|b}$")
    ax.text(0.99, 1.02, f"p97≈{vmax:.2g}", transform=ax.transAxes,
            ha="right", va="bottom", fontsize=8, color="#555")
    fig.tight_layout(); fig.subplots_adjust(bottom=0.18)
    return fig

# --------------------------- UTIL: RUN + SAVE ---------------------------
def export_standardized_assets(mat: pd.DataFrame, fig, H: int, r_disc: float, mode: str):
    """Save standardized PNG + CSVs for Streamlit; keep math unchanged."""
    paths = std_paths(horizon_end=H, r_disc=r_disc, mode=mode)

    # 1) PNG
    fig.savefig(paths["png"], dpi=300, bbox_inches="tight")

    # 2) Wide CSV (Region×Sector matrix)
    wide = mat.copy()
    wide.index.name = "Region"
    wide.to_csv(paths["data_wide"])

    # 3) Long CSV
    long_df = (wide.reset_index()
                    .melt(id_vars="Region", var_name="Sector", value_name="Eq9_Ctilde"))
    long_df.to_csv(paths["data_long"], index=False)

def run_and_export(pdf_path, png_dir, discount_rates, bau_price_spec, title_suffix, mode_key: str):
    """
    mode_key: 'eq9_regional' for dict prices; 'eq9_bau20' for constant price.
    """
    os.makedirs(png_dir, exist_ok=True)
    with PdfPages(pdf_path) as pp:
        # Cover page (unchanged content)
        fig = plt.figure(figsize=(11, 8.5)); plt.axis("off")
        bullets = [
            r"Eq. (9): $\tilde{C}^{z|b} = \frac{\sum_t D_t P^z_{r,t}(E^b-E^z)_+}{\sum_t D_t P^{b,\mathrm{fixed}}_r E^b}$",
            "NZ price: region-level Price|Carbon from PRICES_XLSX.",
            f"Sectors (NGFS): " + ", ".join(SECTOR_ORDER_FIXED),
            f"Horizons: 2020–2030, 2020–2050. Discount rates: " +
            ", ".join([f"{int(r*100)}%" for r in discount_rates]) + f" (t0 = {T0_ANCHOR}).",
        ]
        if isinstance(bau_price_spec, dict):
            bullets.insert(2, "BAU prices: region-specific fixed values.")
        else:
            bullets.insert(2, f"BAU price: fixed = ${float(bau_price_spec):g}/tCO₂ for all regions.")
        y = 0.92
        plt.text(0.05, y, f"Transition Cost — Eq. (9) Heatmaps {title_suffix}",
                 fontsize=16, weight="bold"); y -= 0.06
        for b in bullets:
            plt.text(0.05, y, u"\u2022 " + b, fontsize=11); y -= 0.035
        plt.text(0.05, 0.06,
                 f"Inputs: {os.path.basename(EMISSIONS_XLSX)}, {os.path.basename(PRICES_XLSX)}",
                 fontsize=9, color="#555")
        pp.savefig(fig); plt.close(fig)

        # Pages + standardized exports
        for r_disc in discount_rates:
            for H in HORIZONS:
                mat = compute_eq9_matrix(H, r_disc, bau_price_spec)
                ttl = f"Eq. (9) — Normalized Transition Cost, 2020–{H} (r={int(r_disc*100)}%) {title_suffix}"
                fig = plot_heatmap(mat, ttl, rotate_xticks=True)

                # Legacy PNG (unchanged)
                fn = f"Eq9_{int(r_disc*100)}pct_{H}{'_BAU20' if not isinstance(bau_price_spec, dict) else ''}.png"
                fig.savefig(os.path.join(png_dir, slugify(fn)), dpi=300, bbox_inches="tight")

                # NEW standardized exports for Streamlit (PNG + CSVs)
                export_standardized_assets(mat, fig, H=H, r_disc=r_disc, mode=mode_key)

                # Add to legacy PDF
                pp.savefig(fig)
                plt.close(fig)

    print(f"[OK] PDF -> {pdf_path}\n[OK] PNGs -> {png_dir}")
    print(f"[OK] Standardized exports -> {STD_ROOT}")

# --------------------------- RUN BOTH SCENARIOS ---------------------------
# 1) Main: regional BAU prices, r∈{2%,0%}
run_and_export(
    pdf_path=PDF_OUT_MAIN,
    png_dir=PNG_DIR_MAIN,
    discount_rates=DISCOUNT_RATES_MAIN,
    bau_price_spec=REGIONAL_BAU_FIXED,
    title_suffix="(Regional BAU fixed prices)",
    mode_key="eq9_regional",
)

# 2) Special: BAU fixed = $20/tCO2 for all regions, r=2%
run_and_export(
    pdf_path=PDF_OUT_20,
    png_dir=PNG_DIR_20,
    discount_rates=DISCOUNT_RATES_20,
    bau_price_spec=20.0,
    title_suffix="(BAU fixed = $20/tCO₂)",
    mode_key="eq9_bau20",
)


[OK] PDF -> /Users/noenotter/Documents/Summer_intern/outputs/NGFS_Heatmaps_Transition_Cost.pdf
[OK] PNGs -> /Users/noenotter/Documents/Summer_intern/outputs/NGFS_Eq9_png
[OK] Standardized exports -> /Users/noenotter/Documents/Summer_intern/outputs/transition
[OK] PDF -> /Users/noenotter/Documents/Summer_intern/outputs/NGFS_Heatmaps_Transition_Cost_BAU20_r2pct.pdf
[OK] PNGs -> /Users/noenotter/Documents/Summer_intern/outputs/NGFS_Eq9_BAU20_r2pct_png
[OK] Standardized exports -> /Users/noenotter/Documents/Summer_intern/outputs/transition


In [1]:
# ==========================================================
# Path Misalignment (A), Budget Overshoot (Ω), Abatement Share (AS)
# — calculations/plots unchanged —
# + standardized exports for Streamlit (PNG + CSV wide + CSV long)
# ==========================================================
import os, re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from pathlib import Path

# ---------------- Config ----------------
ROOT = os.path.expanduser("~/Documents/Summer_intern")
EMISSIONS_XLSX = os.path.join(ROOT, "NGFS_GCAM_Carbon_Emissions_Sectors.xlsx")

OUTDIR = os.path.join(ROOT, "outputs"); os.makedirs(OUTDIR, exist_ok=True)
PDF_PATH = os.path.join(OUTDIR, "NGFS_Heatmaps.pdf")
PNG_DIR  = os.path.join(OUTDIR, "NGFS_Heatmaps_png"); os.makedirs(PNG_DIR, exist_ok=True)

# NEW standardized base (what Streamlit app v3 reads)
STD_TRANS_ROOT = Path(ROOT) / "outputs" / "transition"

BAU_SCEN = "Current Policies"
NZ_SCEN  = "Net Zero 2050"

HORIZONS   = [2030, 2050]                 # for Ω and AS
A_WINDOWS  = [(2020, 2030), (2020, 2050)] # for A

# Fixed rows (regions)
REQUESTED_REGIONS = [
    "USA","EU-15","Europe non EU","Japan","China","India","South Korea",
    "Canada","Australia-NZ","Taiwan","Mexico","Russia","Brazil","South Africa","Argentina"
]
REGION_MAP = {"Europe non EU": "Europe_Non_EU", "Australia-NZ": "Australia_NZ"}

# Fixed columns (NGFS exact sector names)
SECTOR_ORDER_FIXED = [
    "Supply","Other Energy Supply","Electricity","Steel","Cement","Chemicals",
    "Other Industry","Industry","Transportation","AFOLU","Other",
]
SUPPLY_COMPONENTS = ["Electricity", "Other Energy Supply"]

# ---------------- Helpers ----------------
def read_emissions_xlsx(path: str) -> pd.DataFrame:
    if not os.path.exists(path): raise FileNotFoundError(path)
    xls = pd.ExcelFile(path)
    if "data" not in xls.sheet_names: raise ValueError(f"'data' not in {xls.sheet_names}")
    df = pd.read_excel(xls, sheet_name="data")
    df["Region"] = df["Region"].astype(str).str.split("|").str[-1]
    return df

def year_columns(df: pd.DataFrame) -> list:
    return [c for c in df.columns if str(c).isdigit()]

def longify(df: pd.DataFrame) -> pd.DataFrame:
    yrs = year_columns(df)
    out = df.melt(
        id_vars=["Model","Scenario","Region","Variable","Unit"],
        value_vars=yrs, var_name="Year", value_name="Value"
    )
    out["Year"] = out["Year"].astype(int)
    return out.dropna(subset=["Value"])

def filter_kyoto(df_long: pd.DataFrame) -> pd.DataFrame:
    return df_long[df_long["Variable"].astype(str).str.startswith("Emissions|Kyoto Gases|")].copy()

# Map NGFS last-token sector names to our exact labels (case-insensitive)
_SECTOR_CANON = {
    "supply": "Supply",
    "other energy supply": "Other Energy Supply",
    "electricity": "Electricity", "power": "Electricity",
    "steel": "Steel",
    "cement": "Cement",
    "chemicals": "Chemicals",
    "other industry": "Other Industry",
    "industry": "Industry",
    "transportation": "Transportation", "transport": "Transportation",
    "afolu": "AFOLU",
    "other": "Other",
}
def map_sector(variable: str) -> str:
    last = str(variable).split("|")[-1].strip().lower()
    return _SECTOR_CANON.get(last, "Other")

def resolve_regions(req: list, region_map: dict, available: set):
    mapped = [region_map.get(r, r) for r in req]
    present = [r for r in mapped if r in available]
    reverse = {region_map.get(k, k): k for k in req}
    return present, reverse

def ensure_sector_columns(mat: pd.DataFrame) -> pd.DataFrame:
    for col in SECTOR_ORDER_FIXED:
        if col not in mat.columns: mat[col] = np.nan
    return mat[SECTOR_ORDER_FIXED]

def slugify(s: str) -> str:
    return re.sub(r"[^A-Za-z0-9._-]+", "_", s).strip("_")

def per_heatmap_vmax(matrix, p=97):
    vals = matrix.values.ravel().astype(float)
    vals = vals[np.isfinite(vals)]
    return max(np.nanpercentile(vals, p), 1e-12) if vals.size else 1.0

# ---------- NEW: standardized export helpers ----------
def std_paths_transition_metric(horizon_end: int, metric_key: str):
    """
    metric_key in {'pm','bo','as'}.
    Returns directories for standardized exports.
    """
    horizon_str = f"2020-{horizon_end}"
    base = STD_TRANS_ROOT / horizon_str / metric_key
    data_dir = base / "data"
    png_dir  = base / "png"
    data_dir.mkdir(parents=True, exist_ok=True)
    png_dir.mkdir(parents=True, exist_ok=True)
    return {
        "data_wide": data_dir / "heatmap_data.csv",
        "data_long": data_dir / "heatmap_data_long.csv",
        "png":       png_dir  / "heatmap.png",
    }

def export_std(mat: pd.DataFrame, fig, horizon_end: int, metric_key: str):
    """Save standardized PNG + CSVs for Streamlit (PM/BO/AS)."""
    paths = std_paths_transition_metric(horizon_end, metric_key)
    # 1) PNG
    fig.savefig(paths["png"], dpi=300, bbox_inches="tight")
    # 2) Wide CSV
    wide = mat.copy()
    wide.index.name = "Region"
    wide.to_csv(paths["data_wide"])
    # 3) Long CSV
    long_df = (wide.reset_index()
                    .melt(id_vars="Region", var_name="Sector", value_name="Value"))
    long_df.to_csv(paths["data_long"], index=False)

# Plot utility (unchanged style)
def plot_heatmap(matrix: pd.DataFrame, title: str, cbar_label: str = "", metric: str = None):
    import seaborn as sns
    fig, ax = plt.subplots(figsize=(0.5*max(8, matrix.shape[1]) + 6,
                                    0.45*max(6, matrix.shape[0]) + 3))
    cm = plt.get_cmap("RdYlGn_r").copy()
    try: cm.set_bad('#e6e6e6')
    except: pass
    vmax = per_heatmap_vmax(matrix, p=97)
    sns.heatmap(matrix.clip(lower=0, upper=vmax), ax=ax, cmap=cm,
                vmin=0, vmax=vmax, mask=np.isnan(matrix.values),
                linewidths=0.2, linecolor='white', cbar=False)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right",
                       rotation_mode="anchor", fontsize=9)
    ax.set_title(title, fontsize=11); ax.set_xlabel("Sector"); ax.set_ylabel("Region")
    im = ax.collections[0]
    cbar = fig.colorbar(im, ax=ax, fraction=0.046, pad=0.02)
    if not cbar_label:
        if metric == "A":       cbar_label = "Path misalignment (A)"
        elif metric == "Omega": cbar_label = "Budget overshoot (Ω)"
        elif metric == "AS":    cbar_label = "Abatement share (AS)"
        else:                   cbar_label = "Value"
    cbar.set_label(cbar_label)
    ax.text(0.99, 1.02, f"p97≈{vmax:.2g}", transform=ax.transAxes,
            ha="right", va="bottom", fontsize=8, color="#555")
    fig.tight_layout(); fig.subplots_adjust(bottom=0.18)
    return fig

# ---------------- Load & prep ----------------
raw = read_emissions_xlsx(EMISSIONS_XLSX)
df_long = filter_kyoto(longify(raw))

# Keep requested regions (intersection, fixed order)
avail_regions = set(df_long["Region"].unique())
regions_order, reverse_map = resolve_regions(REQUESTED_REGIONS, REGION_MAP, avail_regions)
df_long = df_long[df_long["Region"].isin(regions_order)].copy()

# Canonical sectors (exact NGFS names)
df_long["Sector"] = df_long["Variable"].apply(map_sector)
df_long["Year"]   = df_long["Year"].astype(int)

# Per-year series
def per_year_series(df, scen):
    d = df[df["Scenario"]==scen].copy()
    s = d.groupby(["Region","Sector","Year"], as_index=False)["Value"].sum()
    return s.rename(columns={"Value": f"E_{'b' if scen==BAU_SCEN else 'z'}"})

Eb = per_year_series(df_long, BAU_SCEN)
Ez = per_year_series(df_long, NZ_SCEN)
M  = Eb.merge(Ez, on=["Region","Sector","Year"], how="inner")

# If 'Supply' missing but components exist, synthesize it
present_sectors = set(M["Sector"].unique())
if "Supply" not in present_sectors and all(s in present_sectors for s in SUPPLY_COMPONENTS):
    sup = (M[M["Sector"].isin(SUPPLY_COMPONENTS)]
             .groupby(["Region","Year"], as_index=False)[["E_b","E_z"]].sum())
    sup["Sector"] = "Supply"
    M = pd.concat([M, sup], ignore_index=True, sort=False)

# ---------------- Build PDF + PNGs + STANDARD EXPORTS ----------------
with PdfPages(PDF_PATH) as pp:

    # ----- A: Path Misalignment (strict) -----
    for (ymin, ymax) in A_WINDOWS:
        H = ymax  # horizon end for standardized folder
        A = M[(M["Year"]>=ymin) & (M["Year"]<=ymax)].copy()
        A = A[(A["E_b"]>0) & (A["E_z"]>0)].copy()
        A["log_ratio"] = np.log(A["E_b"] / A["E_z"])
        A_sum = (A.groupby(["Region","Sector"], as_index=False)["log_ratio"].sum()
                   .rename(columns={"log_ratio":"A"}))
        A_mat = A_sum.pivot(index="Region", columns="Sector", values="A").reindex(index=regions_order)
        A_mat = ensure_sector_columns(A_mat)

        title = f"Path Misalignment A = Σ ln(E_b/E_z), {ymin}–{ymax}"
        fig = plot_heatmap(A_mat, title, metric="A")
        pp.savefig(fig)
        # legacy PNG (unchanged)
        fig.savefig(os.path.join(PNG_DIR, slugify(f"A_{ymin}-{ymax}.png")), dpi=300, bbox_inches="tight")
        # NEW standardized export for Streamlit
        export_std(A_mat, fig, horizon_end=H, metric_key="pm")
        plt.close(fig)

    # ----- Ω and AS (strict; per-panel dynamic scaling like A) -----
    for H in HORIZONS:
        Z = M[M["Year"] <= H].copy()
        Z["diff_pos"] = np.clip(Z["E_b"] - Z["E_z"], 0.0, None)

        G = (Z.groupby(["Region", "Sector"], as_index=False)
               .agg(Eb_sum=("E_b", "sum"),
                    Ez_sum=("E_z", "sum"),
                    diff_sum=("diff_pos", "sum")))

        # Strict formulas: NaN if denominator <= 0
        G["Omega"] = np.where(G["Ez_sum"] > 0, G["diff_sum"] / G["Ez_sum"], np.nan)
        G["AS"]    = np.where(G["Eb_sum"] > 0, G["diff_sum"] / G["Eb_sum"], np.nan)

        panels = [
            ("Omega", f"Budget Overshoot Ω, 2020–{H}", "bo", "Omega"),
            ("AS",    f"Abatement Share, 2020–{H}",     "as", "AS"),
        ]

        for col, title, mkey, metric in panels:
            mat = (G.pivot(index="Region", columns="Sector", values=col)
                     .reindex(index=regions_order))
            mat = ensure_sector_columns(mat)

            fig = plot_heatmap(mat, title, metric=metric)
            pp.savefig(fig)
            # legacy PNG (unchanged)
            fig.savefig(os.path.join(PNG_DIR, slugify(f"{col}_{H}.png")), dpi=300, bbox_inches="tight")
            # NEW standardized export for Streamlit
            export_std(mat, fig, horizon_end=H, metric_key=mkey)
            plt.close(fig)

print(f"[OK] PDF written -> {PDF_PATH}\n[OK] PNGs in -> {PNG_DIR}")
print(f"[OK] Standardized PM/BO/AS -> {STD_TRANS_ROOT}/2020-{{2030,2050}}/{{pm,bo,as}}/{{data,png}}/")


[OK] PDF written -> /Users/noenotter/Documents/Summer_intern/outputs/NGFS_Heatmaps.pdf
[OK] PNGs in -> /Users/noenotter/Documents/Summer_intern/outputs/NGFS_Heatmaps_png
[OK] Standardized PM/BO/AS -> /Users/noenotter/Documents/Summer_intern/outputs/transition/2020-{2030,2050}/{pm,bo,as}/{data,png}/
